In [2]:
import os
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from numpy import ndarray
from tensorflow import Tensor
from tensorflow.keras.datasets.fashion_mnist import load_data as load_fashion_mnist
from tensorflow.keras.datasets.mnist import load_data as load_mnist
from tensorflow.keras.layers import BatchNormalization as BatchNorm
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Dense, Flatten, Reshape
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.python.keras.utils.layer_utils import count_params

In [3]:
def get_portable_path() -> Path:
    """Utility for getting a sensible working directory whether running as a script or in Colab"""
    try:
        outdir = Path(__file__).resolve().parent
        return outdir
    except NameError:
        print("Possible use of Colab detected. Attempting to exploit `globals()`...")
    try:
        outdir = Path(globals()["_dh"][0]).resolve()
        return outdir
    except KeyError:
        print("Colab not detected.")
        print("Defaulting to current working directory for files.")
        return Path().resolve()

In [10]:
OUTDIR = get_portable_path() / "outputs"
DATADIR = get_portable_path() / "dataset"
BATCH_SIZE = 16

Possible use of Colab detected. Attempting to exploit `globals()`...
Possible use of Colab detected. Attempting to exploit `globals()`...


In [5]:
# Here, we build a naive (inefficient) convolutional autoencoder by simply stacking convolutions
# in the encoding path, and then stacking the appropriate number of transposed convolutions in the
# decoding path. More efficient architectures would employ strided convolutions and/or MaxPooling
# layers, but this make determining the number and size of the transposed convolutions *much* more
# difficult, especially if you want to easily be able to adjust depth. So we settle on the simple
# architecture below.
class ConvAutoencoder(Model):
    def __init__(self, depth: int = 8) -> None:
        super().__init__()
        conv_args = dict(filters=4, kernel_size=3, activation="relu")  # save space / repetition

        # Build an encoding path
        self.encoder = Sequential()
        self.encoder.add(Conv2D(data_format="channels_last", input_shape=IMG_SHAPE, **conv_args))
        self.encoder.add(BatchNorm())
        for _ in range(depth - 1):
            self.encoder.add(Conv2D(**conv_args))
            self.encoder.add(BatchNorm())

        # this line also forces a bottleneck of sorts, in that it forces the code to have 1 channel
        # this is why the encoded representation can be plotted as a black and white images. Note
        # that if you changed below to `filters=3`, then you could see what coloured feature maps
        # would look like.
        self.encoder.add(Conv2D(padding="same", **conv_args))

        # Build a decoding path
        encodeshape = self.encoder.output_shape[1:]
        self.decoder = Sequential()
        self.decoder.add(Conv2DTranspose(padding="same", input_shape=encodeshape, **conv_args))
        self.encoder.add(BatchNorm())
        for _ in range(depth - 1):
            self.decoder.add(Conv2DTranspose(**conv_args))
            self.encoder.add(BatchNorm())
        self.decoder.add(Conv2DTranspose(filters=1, kernel_size=3, activation="linear"))

    def call(self, x: Tensor) -> Tensor:
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train = ImageDataGenerator(rescale = (1./255))

In [13]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  DATADIR,
  validation_split=0.1,
  subset="training",
  seed=123,
  image_size=(300, 300),
  batch_size=BATCH_SIZE)

Found 3141 files belonging to 3 classes.
Using 2827 files for training.


2021-11-30 10:37:45.719272: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-11-30 10:37:46.482239: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-11-30 10:37:46.483226: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-11-30 10:37:46.494866: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

In [ ]:
dir(train_ds)

Error: Session cannot generate requests